In [1]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group 
#add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [2]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
#add_all_category_columns(df) # Add '_c' categorical columns

In [3]:
# Function to creaet each section of the standard report. 
def standard_report_section(df, column_name, totalcount, totalamount):
    dfo = df.groupby(column_name)
    dfo = dfo['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    dfo['%count'] = np.round(100*dfo['count']/totalcount,1)
    dfo['%amount'] = np.round(100*dfo['amount']/totalamount,1)
    
    return dfo[['median','count','%count','amount','%amount']]

    

In [4]:
#
# Calling the standard_report_section

totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

# List Version
frames = []
for col_name in ('purpose_type','race_eth','income_group'  ):
    frames.append(standard_report_section(df, column_name, totalcount, totalamount))
  
# Comprehension version

frames = [standard_report_section(df, column_name, totalcount, totalamount) for column_name in 
         ('purpose_type','race_eth','income_group'  )]
    
table = pd.concat(frames, keys=['Loan Purpose and Type', 
                               'Applicant Race/Eth', 
                               'Applicant Income',
                               'Applicant Sex',
                               'Occupancy'])
pieces = [table.ix[:, ["median","count","%count","amount","%amount",]]]
table = pd.concat(pieces, axis=1, ) #keys=[header] for top header row

table

NameError: name 'column_name' is not defined

In [ ]:
def standardReport(newdf): # (newdf, header) 
    totalcount = newdf.loan_amount.count()
    totalamount = newdf.loan_amount.sum()
    ### By Loan Purpose and Type
    loanpt = newdf.groupby(['purpose_type'])
    loanpt = loanpt['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
    loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
    loanpt=loanpt[['median','count','%count','amount','%amount']]
    ### Subtotals
    
    ### By Applicant Race/Ethnicity
    race_eth  = newdf.groupby(['race_eth'])
    race_eth  = race_eth ['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
    race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
    race_eth = race_eth [['median','count','%count','amount','%amount']]
    ###By Applicant Income
    income  = newdf.groupby(['income_group'])
    income   = income['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    income ['%count'] = np.round(100*income['count']/totalcount,1)
    income ['%amount'] = np.round(100*income['amount']/totalamount,1)
    income = income [['median','count','%count','amount','%amount']]
    ###By Applicant Sex
    sex  = newdf.groupby(['applicant_sex'])
    sex   = sex['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    sex ['%count'] = np.round(100*sex['count']/totalcount,1)
    sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
    sex = sex [['median','count','%count','amount','%amount']]
    ###Owner-Occupied
    occ = newdf.groupby(['occupancy'])
    occ   = occ['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    occ ['%count'] = np.round(100*occ['count']/totalcount,1)
    occ ['%amount'] = np.round(100*occ['amount']/totalamount,1)
    occ = occ [['median','count','%count','amount','%amount']]
  
    ### Smashing Tables Together
    frames = [loanpt, race_eth,income,sex,occ]
    
    table = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex',
                                   'Occupancy'])
    pieces = [table.ix[:, ["median","count","%count","amount","%amount",]]]
    table = pd.concat(pieces, axis=1, ) #keys=[header] for top header row
    return table;

# All Lenders Combined
### Standard Report

In [ ]:
standardReport(df)

In [ ]:
standardReport(df).to_csv('standardReport.csv')